In [12]:
import pandas as pd
import numpy as np
import duckdb
import warnings
import os

import xgboost as xgb
from sklearn.preprocessing import LabelEncoder

# email
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart

import warnings
from datetime import datetime, timedelta

pd.set_option('display.max_columns', None)
warnings.filterwarnings("ignore")

categories = ['PTS', 'AST', 'REB', 'PR', 'PA', 'RA', 'PRA', 'TPM', 'STL', 'BLK', 'STL_BLK']
con = duckdb.connect(database=":memory:")

cwd = os.path.abspath(os.getcwd()).replace("\\", "/")
if cwd.startswith("C:/Users/Rodolfo/"):
    RUN_LOCATION = "local"
else:
    RUN_LOCATION = "cloud"
time_offset = {"local": 3, "cloud": -5}
now = str((datetime.now() + timedelta(hours=time_offset[RUN_LOCATION]) + timedelta(hours=-3)).date())
print(f"Today's date:", now)

Today's date: 2026-01-10


In [2]:
%run ./common_utils.ipynb

# Initial Functions

In [3]:
def email(model, error):
    
    # Email details
    sender_email = "rodolfoe7157@gmail.com"
    receiver_email = "rodolfoe7157@gmail.com"
    password = "cqgu bfey cnyx sfue"  # See note below

    subject = "NBA create_Predictions error"
    body = f"Model: {model}_model\nERROR: {error}"

    # Create message
    msg = MIMEMultipart()
    msg['From'] = sender_email
    msg['To'] = receiver_email
    msg['Subject'] = subject
    msg.attach(MIMEText(body, 'plain'))

    # Connect to Gmail SMTP server and send
    with smtplib.SMTP_SSL('smtp.gmail.com', 465) as server:
        server.login(sender_email, password)
        server.send_message(msg)

    print("Email sent successfully!")

In [4]:
def load_df(file_name):
    df = pd.DataFrame()
    for i in [2021, 2022, 2023, 2024, 2025]:
        df_temp = pd.read_csv(f"../tables/{i}/{file_name}.csv")
        df_temp['Season'] = i
        df = pd.concat([df, df_temp])
        
    if 'Date' in df.columns:
        df['Date'] = pd.to_datetime(df.Date)
    if file_name == "season_gamelogs":
        df = df[~df[['Date', 'Team', 'Player']].duplicated(keep='last')]
    
    return df

In [5]:
def create_base_df():
    
    # Load dfs
    df = load_df('parlay_stats')
    df2 = load_df('nba_schedule')
    df3 = load_df('season_gamelogs')
    df4 = load_df('injuries')
    df5 = load_df('plyr_pos_xref')
    df6 = load_df('daily_lineups')
    gmlog_cols = ['game_id', 'Player', 'MP', 'PF']
    df7 = load_df('h1_season_gamelogs')[gmlog_cols].rename(columns={"MP": "MP_h1", "PF": "PF_h1"})
    df8 = load_df('h2_season_gamelogs')[gmlog_cols].rename(columns={"MP": "MP_h2", "PF": "PF_h2"})
    df9 = load_df('q1_season_gamelogs')[gmlog_cols].rename(columns={"MP": "MP_q1", "PF": "PF_q1"})
    df10 = load_df('q2_season_gamelogs')[gmlog_cols].rename(columns={"MP": "MP_q2", "PF": "PF_q2"})
    df11 = load_df('q3_season_gamelogs')[gmlog_cols].rename(columns={"MP": "MP_q3", "PF": "PF_q3"})
    df12 = load_df('q4_season_gamelogs')[gmlog_cols].rename(columns={"MP": "MP_q4", "PF": "PF_q4"})

    df3 = df3.rename(columns={"3PM": "TPM", "3PA": "TPA", "3P%": "TP%", "TRB": "REB"}).drop(['Pos', 'Opp'], axis=1)
    df3['PR'] = df3.PTS + df3.REB 
    df3['PA'] = df3.PTS + df3.AST
    df3['RA'] = df3.REB + df3.AST
    df3['PRA'] = df3.PTS + df3.REB + df3.AST
    df3['STL_BLK'] = df3.STL + df3.BLK
    df = df.merge(df3, on=['Season', 'Date', 'Team', 'Player'], how='left')

    df_mtch = df2[['Season', 'Date', 'AwayABV', 'HomeABV', 'AwayPTS', 'HomePTS', 'AwayB2B', 'HomeB2B', 'is_OT', 'cup_gm', 'pstszn_gm']]
    df_mtch['Team_type'] = 'Away'
    df_mtch = df_mtch.rename(columns={"AwayABV": "Team", "HomeABV": "Opp", "AwayB2B": "B2B"})[['Season', 'Date', 'Team', 'AwayPTS', 'HomePTS', 'Opp', 'B2B', 'is_OT', 'cup_gm', 'pstszn_gm', 'Team_type']]
    df_mtch2 = df_mtch.copy().rename(columns={"Team": "Opp", "Opp": "Team", "HomeB2B": "B2B"})[['Season', 'Date', 'Team', 'AwayPTS', 'HomePTS', 'Opp', 'B2B', 'is_OT', 'cup_gm', 'pstszn_gm']]
    df_mtch2['Team_type'] = 'Home'
    df_mtch = pd.concat([df_mtch, df_mtch2])
    df_mtch = df_mtch[['Season', 'Date', 'Team', 'Team_type', 'AwayPTS', 'HomePTS', 'is_OT', 'cup_gm', 'pstszn_gm']]
    df_mtch = df_mtch.sort_values(["Team", "Date"])
    df_mtch['team_game_num'] = df_mtch.groupby(["Team", "Season"]).cumcount() + 1
    df_mtch['Spread'] = np.where(df_mtch.Team_type == 'Home', df_mtch.AwayPTS - df_mtch.HomePTS, df_mtch.HomePTS - df_mtch.AwayPTS)
    df_mtch['Total'] = df_mtch.AwayPTS + df_mtch.HomePTS
    df_mtch['is_Win'] = np.where(df_mtch.Spread > 0, 1, 0)
    df_mtch['Szn_Wins'] = df_mtch.groupby(['Season', 'Team'])['is_Win'].cumsum()
    df = df.drop(['Season', 'Team_type'], axis=1).merge(df_mtch, on=['Date', 'Team'])

    df = df.merge(df4[['Date', 'Team', 'Player', 'Status']], on=['Date', 'Team', 'Player'], how='left')
    df['Status'] = np.where((df.Active == 1) & (df.Status.isnull()), 'Available', df.Status)
    df['Status'] = np.where((df.Active == 0), 'Out', df.Status)
    df['Status'] = np.where((df.Status == 'Out') & (df.Active != 0), 'Available', df.Status)

    df6['role'] = 1
    df = df.merge(df6.drop('Pos', axis=1), on=['Season', 'Date', 'Team', 'Player'], how='left')
    df['role'] = df.role.fillna(2).astype(int)
    df['role'] = np.where(((df.MP < 8) & (df.role == 2)), 3, df.role)

    # Add gmlog splits
    df_gmlog_comb = df7.merge(df8, on=['game_id', 'Player'])
    for df_loop in (df9, df10, df11, df12):
        df_gmlog_comb = df_gmlog_comb.merge(df_loop, on=['game_id', 'Player'])
    df = df.merge(df_gmlog_comb, on=['game_id', 'Player'], how='left')
    
    global team_encoder, player_encoder, team_type_encoder, position_encoder, status_encoder
    team_encoder = LabelEncoder()
    player_encoder = LabelEncoder()
    team_type_encoder = LabelEncoder()
    position_encoder = LabelEncoder()
    status_encoder = LabelEncoder()

    # Encode string cols
    team_encoder.fit(pd.concat([df["Team"], df["Opp"]], axis=0))
    players_fits = pd.concat([df["Player"], df3["Player"]], axis=0)
    players_fits = pd.concat([players_fits, df4["Player"]], axis=0).drop_duplicates()
    player_encoder.fit(players_fits)
    df["Team"] = team_encoder.transform(df["Team"])
    df["Opp"] = team_encoder.transform(df["Opp"])
    df["Player_name"] = df.Player
    df["Player"] = player_encoder.transform(df["Player"])
    df["Pos"] = position_encoder.fit_transform(df["Pos"])
    df['Team_type'] = team_type_encoder.fit_transform(df['Team_type'])
    df["Status"] = status_encoder.fit_transform(df["Status"])

    return df

### Create missing_df

In [6]:
def create_df_missing(df, pred_col):
    
    df3 = load_df('season_gamelogs')
    df3 = df3.rename(columns={"3PM": "TPM", "3PA": "TPA", "3P%": "TP%", "TRB": "REB"}).drop(['Pos', 'Opp'], axis=1)
    df3['PR'] = df3.PTS + df3.REB 
    df3['PA'] = df3.PTS + df3.AST
    df3['RA'] = df3.REB + df3.AST
    df3['PRA'] = df3.PTS + df3.REB + df3.AST
    df3['STL_BLK'] = df3.STL + df3.BLK
    df4 = load_df('injuries')
    # Fill missing games from injuries.csv
    df_pred = create_base_df()
    team_games = df_pred[['Season', 'Team', 'Date']].drop_duplicates()
    players = df_pred[['Season','Player','Team']].drop_duplicates()
    fabricated = (players.sort_values('Season').groupby('Player', as_index=False).last())
    fabricated['Season'] = fabricated['Season'] + 1
    players = pd.concat([players, fabricated], ignore_index=True).drop_duplicates(['Season','Player','Team'])
    expanded = team_games.merge(players, on=['Season', 'Team'], how='left')

    df3["Team"] = team_encoder.transform(df3["Team"])
    df3["Player"] = player_encoder.transform(df3["Player"])
    df4["Team"] = team_encoder.transform(df4["Team"])
    df4["Player"] = player_encoder.transform(df4["Player"])
    df5 = load_df('plyr_pos_xref')
    df5['Team'] = team_encoder.transform(df5["Team"])
    df5['Player'] = player_encoder.transform(df5["Player"])

    expanded = expanded.merge(df3[['Season', 'Player', 'Date', 'MP']], on=['Season', 'Player', 'Date'], how='left').drop_duplicates(['Season', 'Date', 'Player', 'Team'])
    expanded = expanded[(expanded.MP.isnull()) & (expanded.Date != now)].drop('MP', axis=1)
    expanded = pd.concat([expanded, df4[df4.Status == 'Out'][['Season', 'Team', 'Date', 'Player']]])
    df4 = df4.merge(expanded, on=['Season', 'Date', 'Team', 'Player'], how='right')

    # Grab outs from players season gamelogs
    df4 = df4.merge(df3, on=['Season', 'Date', 'Team', 'Player'], how='outer')
    df4['Status'] = np.where(((df4.Active == 1) | (df4.MP > 0)), 'Available', df4.Status)
    df4['Status'] = np.where(((df4.Active == 0) | (df4.MP == 0) | (df4.MP.isnull())), 'Out', df4.Status)
    df4['Status'] = np.where((df4.Status == 'Out') & (df4.MP > 0), 'Available', df4.Status)
    df4['Status'] = np.where((df4.Status != 'Out') & (df4.MP == 0), 'Out', df4.Status)
    df4 = df4[df4.Status == 'Out'][['Season', 'Date', 'Team', 'Player']].drop_duplicates()
    
    df_missing = df[['Season', 'Date', 'Team', 'Player', 'role', pred_col]].copy()
    df_missing[f'{pred_col}_L10'] = (
        df_missing.groupby(['Player','Season'])[pred_col]
                  .transform(lambda x: x.rolling(10, min_periods=1).mean())
    )
    df_missing['role_L10_mode'] = (
        df_missing
            .groupby(['Player', 'Season'])['role']
            .transform(lambda x: x.rolling(10, min_periods=1)
                            .apply(lambda y: np.bincount(y.astype(np.int8), minlength=4).argmax(), raw=True))
    )
    df_missing = pd.merge_asof(df4, df_missing[["Season", "Player", "Date", "role", "role_L10_mode", f"{pred_col}_L10"]], 
                      on="Date", by=["Player", "Season"], direction="backward", allow_exact_matches=True).dropna()   
    df_missing = df_missing.merge(df5, on=['Season', 'Team', 'Player'])
    
    # Filter out old injuries
    df_missing = df_missing.sort_values(["Season", "Team", "Player", "Date"])
    df_missing["team_game_num"] = (df_missing.groupby(["Season", "Team"])["Date"].rank(method="dense").astype(int))
    df_missing["game_break"] = (df_missing.groupby(["Season", "Team", "Player"])["team_game_num"].diff().ne(1))
    df_missing["streak_id"] = (df_missing.groupby(["Season", "Team", "Player"])["game_break"].cumsum())
    df_missing["consecutive_games"] = (df_missing.groupby(["Season", "Team", "Player", "streak_id"]).cumcount().add(1))
    df_missing["eligible_today"] = (df_missing["consecutive_games"] < 10).astype(int)
    df_missing["role_for_count"] = np.where(df_missing["eligible_today"] == 1, df_missing["role_L10_mode"], np.nan)    

    df_missing["Player"] = player_encoder.inverse_transform(df_missing["Player"])
#     display(df_missing[df_missing.Team == 7].tail(50))

    out_minutes = (
    df_missing
      .groupby(["Season", "Date", "Team"])
      .agg(
#           team_mins_available=("MP_L10", lambda x: x.sum()),
          starters_out=("role_for_count", lambda x: (x == 1).sum())
      )
      .reset_index()
    )

    return out_minutes

# Minutes Projection Model

In [7]:
def setup_df_mins(con, df):

    df = df[['Season', 'Date', 'Team', 'Team_type', 'Opp', 'Player', 'Pos', 'role', 'B2B', 
             'MP', 'MP_h1', 'MP_h2', 'MP_q1', 'MP_q2', 'MP_q3', 'MP_q4', 
             'Spread', 'team_game_num', 'pstszn_gm', 'is_OT']]
    
    for col in ['MP']:
        for N in [3, 5, 10]:
            df[f'{col}_L{N}_avg'] = (
                df.groupby(['Player', 'Season'])[col]
                  .rolling(window=N, min_periods=1)
                  .mean()
                  .shift(1)
                  .reset_index(level=[0, 1], drop=True)
            )
            df[f'prev_team_mins_pct_L{N}'] = df[f'{col}_L{N}_avg'] / 240

    games_last_7_days = df.sort_values(['Player', 'Season', 'Date']).groupby(['Player', 'Season']).rolling('7D', on='Date', closed='left')['MP'].count().reset_index().rename(columns={"MP": "gms_L7_days"})
    games_last_7_days = games_last_7_days.drop_duplicates(
        subset=['Player', 'Season', 'Date']
    )
    df = df.merge(games_last_7_days, on=['Player', 'Season', 'Date'])
    df['gms_L7_days'] = df.gms_L7_days.fillna(0).astype(int)
        
    df['OT_adj_MP'] = np.where(df.is_OT != 0, df.MP + (5 * df.is_OT), df.MP)
    df['role'] = np.where(((df.OT_adj_MP >= 24) & (df.role != 1)), 1, df.role)
    df['role'] = np.where(((df.OT_adj_MP < 24) & (df.role == 1)), 2, df.role)
    df['role'] = np.where(((df.OT_adj_MP < 14) & (df.role == 2)), 3, df.role)
    for N in [1, 3, 5]:
        df[f"recent_role_L{N}"] = (
            df
            .groupby(["Player", "Season"])["role"]
            .rolling(5, min_periods=1)
            .apply(lambda arr: np.bincount(arr.astype(int), minlength=4).argmax(), raw=True)
            .reset_index(level=[0, 1], drop=True)
        )
    df['missed_games'] = (df.groupby(['Player', 'Team', 'Season'])['team_game_num'].diff().sub(1).fillna(0).astype(int))
    
    df['game_spread_type'] = 0
    df['game_spread_type'] = np.where(abs(df.Spread) < 7, 1, df.game_spread_type) 
    df['game_spread_type'] = np.where((abs(df.Spread) >= 7) & (abs(df.Spread) <= 12), 2, df.game_spread_type) 
    df['game_spread_type'] = np.where(abs(df.Spread) > 12, 3, df.game_spread_type) 
    df['game_spread_type'] = np.where(df.is_OT > 0, 1, df.game_spread_type) 

    # Tell model games exist after players injuries/susp
    team_games = df[['Season', 'Team', 'Date', 'team_game_num']].drop_duplicates()
    players = df[['Season','Player','Team']].drop_duplicates()
    fabricated = (players.sort_values('Season').groupby('Player', as_index=False).last())
    fabricated['Season'] = fabricated['Season'] + 1
    players = pd.concat([players, fabricated], ignore_index=True).drop_duplicates(['Season','Player','Team'])
    expanded = team_games.merge(players, on=['Season', 'Team'], how='left')
    expanded = expanded.merge(df[['Season', 'Player', 'Date', 'MP']], on=['Season', 'Player', 'Date'], how='left').drop_duplicates(['Season', 'Date', 'Player', 'Team'])
    expanded['player_played'] = expanded['MP'].notna().astype(int)
    expanded['team_played_no_player'] = ((expanded['player_played'] == 0)).astype(int)
    expanded['tm_plays_after'] = (expanded.groupby(['Player'])['team_played_no_player'].shift(-1))
    expanded['missed_gms_aftr'] = 0
    expanded['missed_gms_aftr'] = np.where((expanded.player_played == 1) & (expanded.tm_plays_after == 1), 1, expanded.missed_gms_aftr)
    df = df.merge(expanded[['Date', 'Team', 'Player', 'missed_gms_aftr']], on=['Date', 'Team', 'Player'])
    
    df2 = create_df_missing(df, 'MP')
    df = df.merge(df2, on=["Season", "Date", "Team"], how='left')
#     df['starters_out'] = df.starters_out.fillna(0)

    df['MP_change_pct_L10'] = (df['MP'] - df['MP_L10_avg']) / df['MP_L10_avg']
    MP_Inc_conds = (
                    ((df.role == 1) & (df['MP_change_pct_L10'] >= 0.05)) | 
                    ((df.role == 2) & (df['MP_change_pct_L10'] >= 0.10)) | 
                    ((df.role == 3) & (df['MP_change_pct_L10'] >= 0.15)) |
                    ((df.role != 3) & (df.starters_out > 2))
#                     ((df.role == 1) & (df.starters_out > 1)) | 
#                     ((df.role == 2) & (df.starters_out > 0))
                   )
    df['MP_Increase'] = np.where(MP_Inc_conds, 1, 0)
    Early_Stop_conds = (
        ((df.role == 1) & (df['MP_change_pct_L10'] <= -0.25)) | 
        ((df.role == 2) & (df['MP_change_pct_L10'] <= -0.35)) | 
        ((df.role == 3) & (df['MP_change_pct_L10'] <= -0.45)) | 
        ((df.role != 3) & (df['MP'] < 8)) | 
        ((df.role == 1) & (df.MP_q4 == 0))
    )
    df['Early_Stop'] = np.where(Early_Stop_conds, 1, 0)
    Early_exit_conds = (
        ((df.MP_q2 + df.MP_h2) == 0 & (df.role == 1))
    )
    df['Early_Exit'] = np.where(Early_exit_conds, 1, 0)
    
    df['Injured'] = ((df.Early_Stop == 1) & (df.missed_gms_aftr > 0)).astype(int)
    df['return_game'] = ((df.groupby('Player')['Injured'].shift(1) == 1) & (df.missed_games > 0)).astype(int)
    df['games_since_return'] = (df.groupby('Player')['return_game'].cumsum())
    df['games_since_return'] = (df.groupby(['Player', 'games_since_return']).cumcount())
    df['ramp_phase'] = 0
    df.loc[df.games_since_return == 0, 'ramp_phase'] = 1
    df.loc[df.games_since_return.isin([1, 2, 3]), 'ramp_phase'] = 2

    df = df.drop(['Season', 'Team_type', 'team_game_num', 'Spread',  
                  'MP_h1', 'MP_h2', 'MP_q1', 'MP_q2', 'MP_q3', 'MP_q4', 'OT_adj_MP', 
                  'missed_gms_aftr', 'games_since_return', 'return_game', 'MP_change_pct_L10'], axis=1)      
    return df

# Main Model

In [8]:
def setup_df_main(df, tgt_stat):
    
    # Stat dependent features 
    if tgt_stat == 'PTS':
        tgt_stat_cols = ['TPM', 'FG', 'FT', 'TPA', 'FGA', 'FTA']
        df = df[['Season', 'Date', 'Team', 'Opp', 'Player', 'Pos', 'MP', 'team_game_num', 
         'PTS', 'TPM', 'FG', 'FGA', 'TPA', 'FT', 'FTA', 
         f'Off_{tgt_stat}', f'Off_L3_{tgt_stat}', f'Off_L5_{tgt_stat}', f'Off_L10_{tgt_stat}', f'Off_{tgt_stat}_Rk',
         f'Def_{tgt_stat}', f'Def_L3_{tgt_stat}', f'Def_L5_{tgt_stat}', f'Def_L10_{tgt_stat}', f'Def_{tgt_stat}_Rk',
         'Spread', 'Total', 'is_OT']]
        
        # Efficiency metrics
        df['three_rate_raw'] =  np.where(df.FGA > 0, df['TPA'] / df['FGA'], 0)
        df['ft_rate_raw']    =  np.where(df.FGA > 0, df['FTA'] / df['FGA'], 0)
        df['eFG_raw'] = (df['FG'] + 0.5 * df['TPM']) / df['FGA']
        df['TS_raw'] = df['PTS'] / (2 * (df['FGA'] + 0.44 * df['FTA']))    
        df['usage_proxy_raw'] =  np.where(df.MP > 0, (df['FGA'] + 0.44 * df['FTA']) / df['MP'], 0)
        
        for w in [3, 5, 10]:
            for metric in ['three_rate', 'ft_rate', 'eFG', 'TS', 'usage_proxy']:
                col = f"{metric}_L{w}"
                df[col] = (
                    df.groupby(['Player','Season'])[f'{metric}_raw']
                      .rolling(w, min_periods=1)
                      .mean()
                      .shift(1)
                      .reset_index(level=[0,1], drop=True)
                )
        for metric in ['three_rate', 'ft_rate', 'eFG', 'TS', 'usage_proxy']:
            col = f'{metric}_weighted'
            df[col] = (
                0.6 * df[f'{metric}_L3'] +
                0.3 * df[f'{metric}_L5'] +
                0.1 * df[f'{metric}_L10']
            )
            df = df.drop(f'{metric}_raw', axis=1)
        
    elif tgt_stat == 'PRA':
        tgt_stat_cols = ['PTS', 'REB', 'AST', 'TPM', 'FG']
        df = df[['Season', 'Date', 'Team', 'Opp', 'Player', 'Pos', 'MP', 'team_game_num', 
         'PTS', 'AST', 'REB', 'PR', 'PA', 'RA', 'PRA', 'TPM', 'STL', 'BLK', 'STL_BLK', 
         'FG', 'FGA', 'TPA', 'FT', 'FTA', 
         f'Off_{tgt_stat}', f'Off_L3_{tgt_stat}', f'Off_L5_{tgt_stat}', f'Off_L10_{tgt_stat}', f'Off_{tgt_stat}_Rk',
         f'Def_{tgt_stat}', f'Def_L3_{tgt_stat}', f'Def_L5_{tgt_stat}', f'Def_L10_{tgt_stat}', f'Def_{tgt_stat}_Rk',
         'Spread', 'Total', 'is_OT']]
        
        df['usage_proxy_raw'] =  np.where(df.MP > 0, (df['FGA'] + 0.44 * df['FTA']) / df['MP'], 0)
        for w in [3, 5, 10]:
            df[f"usage_proxy_L{w}"] = (
                df.groupby(['Player','Season'])[f'usage_proxy_raw']
                  .rolling(w, min_periods=1)
                  .mean()
                  .shift(1)
                  .reset_index(level=[0,1], drop=True)
            )
        df['usage_proxy_weighted'] = (
            0.6 * df[f'usage_proxy_L3'] +
            0.3 * df[f'usage_proxy_L5'] +
            0.1 * df[f'usage_proxy_L10']
        )
        df = df.drop('usage_proxy_raw', axis=1)
        
        
    else:
        tgt_stat_cols = []
        df = df[['Season', 'Date', 'Team', 'Opp', 'Player', 'Pos', 'MP', 'team_game_num', 
         'PTS', 'AST', 'REB', 'PR', 'PA', 'RA', 'PRA', 'TPM', 'STL', 'BLK', 'STL_BLK',
         'FG', 'FGA', 'TPA', 'FT', 'FTA', 
          f'Off_{tgt_stat}', f'Off_L3_{tgt_stat}', f'Off_L5_{tgt_stat}', f'Off_L10_{tgt_stat}', f'Off_{tgt_stat}_Rk',
          f'Def_{tgt_stat}', f'Def_L3_{tgt_stat}', f'Def_L5_{tgt_stat}', f'Def_L10_{tgt_stat}', f'Def_{tgt_stat}_Rk',
         'Spread', 'Total', 'is_OT']]

    
    # Create rolling + lag features    
    for col in ['MP'] + tgt_stat_cols:
        df[f'{col}_lst_gm'] = (
            df
            .groupby(['Player', 'Season'])[col]
            .shift(1)
        )
        for N in [1, 3, 5, 10]:
            df[f'{col}_L{N}_avg'] = (
                df.groupby(['Player', 'Season'])[col]
                  .rolling(window=N, min_periods=1)
                  .mean()
                  .shift(1)
                  .reset_index(level=[0, 1], drop=True)
            )

    # Role identifiers features
    df['reserve_td'] = (df.MP < 8).astype(int)
    df['bench_td']   = ((df.MP >= 8) & (df.MP <= 25)).astype(int)
    df['starter_td'] = (df.MP > 25).astype(int)
    role_counts = df.groupby(['Season', 'Player'])[['reserve_td', 'bench_td', 'starter_td']].sum()
    role_counts['most_common_role'] = role_counts[['reserve_td', 'bench_td', 'starter_td']].idxmax(axis=1)
    role_counts['reserve'] = (role_counts['most_common_role'] == 'reserve_td').astype(int)
    role_counts['bench']   = (role_counts['most_common_role'] == 'bench_td').astype(int)
    role_counts['starter'] = (role_counts['most_common_role'] == 'starter_td').astype(int)
    df = df.merge(role_counts[['reserve', 'bench', 'starter']], on=['Season', 'Player'], how='left')
    df['role'] = 0
    df['role'] = np.where(df.starter == 1, 1, df.role)
    df['role'] = np.where(df.bench == 1, 2, df.role)
    df['role'] = np.where(df.reserve == 1, 3, df.role)
    
    for N in [1, 3, 5]:
        for role in ['reserve_td', 'bench_td', 'starter_td']:
            df[f'{role}_last{N}'] = (
                df.sort_values(['Player', 'Date']).groupby('Player')[role]
                  .rolling(N, min_periods=1)
                  .sum()
                  .shift(1)
                  .reset_index(0, drop=True)
            )
        rec_role_cols = [f'{role}_last{N}' for role in ['reserve_td', 'bench_td', 'starter_td']]
        df[f'recent_most_common_role_L{N}'] = df[rec_role_cols].idxmax(axis=1)
        df[f'recent_role_L{N}'] = 0
        df[f'recent_role_L{N}'] = np.where(df[f'recent_most_common_role_L{N}'] == f'starter_td_last{N}', 1, df[f'recent_role_L{N}'])
        df[f'recent_role_L{N}'] = np.where(df[f'recent_most_common_role_L{N}'] == f'bench_td_last{N}', 2, df[f'recent_role_L{N}'])
        df[f'recent_role_L{N}'] = np.where(df[f'recent_most_common_role_L{N}'] == f'reserve_td_last{N}', 3, df[f'recent_role_L{N}'])      
        df = df.drop(f'recent_most_common_role_L{N}', axis=1)
        for role in ['reserve_td', 'bench_td', 'starter_td']:
            df = df.drop(f'{role}_last{N}', axis=1)
    
    df['game_spread_type'] = 0
    df['game_spread_type'] = np.where(abs(df.Spread < 6), 1, df.game_spread_type) 
    df['game_spread_type'] = np.where((abs(df.Spread >= 6) & abs(df.Spread <= 14)), 2, df.game_spread_type) 
    df['game_spread_type'] = np.where(abs(df.Spread > 14), 3, df.game_spread_type) 
    
    for col in categories + ['Season', 'FG', 'FGA', 'FT', 'FTA', 'TPM', 'TPA', 
                             'reserve_td', 'reserve', 'bench_td', 'bench', 'starter_td', 'starter'] + tgt_stat_cols:
        if col == tgt_stat:
            continue
        if col in df.columns:
            df = df.drop(col, axis=1)
        
    return df

### Today's predictions

In [9]:
def generate_predictions(tgt_stat):
    
    df_pred = create_base_df()
        
    mins_model = xgb.XGBRegressor()
    mins_model.load_model("../ML_models/mins_model.json")
    stat_model = xgb.XGBRegressor()
    stat_model.load_model(f"../ML_models/{tgt_stat}_model.json")
    
    df_lines = pd.read_csv(f"../tables/2025/parlay_lines.csv")
    df_lines['Date'] = pd.to_datetime(df_lines.Date)
    df_lines = df_lines[~(df_lines.Team.isnull())]

    # Predict Mins
    df_lines["Team"] = team_encoder.transform(df_lines["Team"])
    df_pred = df_pred.merge(df_lines[['Date', 'Team', 'Spread', 'Total']], on=['Date', 'Team'], how='left')
    df_pred = df_pred[~df_pred[['Date', 'Team', 'Player']].duplicated(keep='last')]
    df_pred['Spread_x'] = np.where(df_pred.Spread_x.isnull(), df_pred.Spread_y, df_pred.Spread_x)
    df_pred['Total_x'] = np.where(df_pred.Total_x.isnull(), df_pred.Total_y, df_pred.Total_x)
    df_pred = df_pred.rename(columns={"Spread_x": "Spread", "Total_x": "Total"}).drop(['Spread_y', 'Total_y'], axis=1)
    df_pred_mins = setup_df_mins(con, df_pred)
    df_pred_mins = df_pred_mins.drop(['Date', 'MP'], axis=1)
    df_pred['MP'] = mins_model.predict(df_pred_mins)

    # Predict Stat
    df_pred = setup_df_main(df_pred, tgt_stat)
    feature_cols = [col for col in df_pred.columns if col not in ['Date', tgt_stat]]
    df_pred = df_pred[df_pred.Date == now][feature_cols]
    df_pred[f"{tgt_stat}_proj"] = stat_model.predict(df_pred)

    # Setup results
    df_pred['Team'] = team_encoder.inverse_transform(df_pred["Team"])
    df_lines['Team'] = team_encoder.inverse_transform(df_lines["Team"])
    df_pred['Opp'] = team_encoder.inverse_transform(df_pred["Opp"])
    df_pred['Player'] = player_encoder.inverse_transform(df_pred["Player"])
    df_pred['Pos'] = position_encoder.inverse_transform(df_pred["Pos"])

    df_lines = df_lines[df_lines.Date == now][['Team', 'Player', f'{tgt_stat}_line']]
    df_pred = df_pred.merge(df_lines, on=['Team', 'Player'])
    df_pred = df_pred[~(df_pred[f'{tgt_stat}_line'].isnull())]
    df_pred['Diff'] = abs((df_pred[f'{tgt_stat}_line'] - df_pred[f'{tgt_stat}_proj']))
    df_pred['Diff2'] = abs((df_pred['MP'] - df_pred['MP_L5_avg']))
    df_pred = df_pred.sort_values('Diff', ascending=False).drop(['Diff', 'Diff2'], axis=1)
    
#     if df_pred.shape[0] >= 50:
#         print(df_pred.shape[0], 'rows')
#         for tm in df_pred.Team.unique():
#             display(df_pred[df_pred.Team == tm])
#     else:
#         display(df_pred)

    tds_picks = df_pred[['Team', 'Player', 'Pos', 'Opp', 'MP', 'MP_L5_avg', 'Spread', f'{tgt_stat}_line', f'{tgt_stat}_proj']]
    if tds_picks.shape[0] >= 50:
        print(tds_picks.shape[0], 'rows')
        for tm in tds_picks.Team.unique():
            display(tds_picks[tds_picks.Team == tm])
    else:
        display(tds_picks)
    tds_picks.insert(0, 'Date', pd.to_datetime(now))
    partition_save_df(tds_picks, f"../tables/2025/gmday_preds_{tgt_stat}.csv")

In [10]:
try: 
    generate_predictions('PTS')
except Exception as e:
    email('PTS', e)
    raise Exception(e)

130 rows


,Team,Player,Pos,Opp,MP,MP_L5_avg,Spread,PTS_line,PTS_proj
62,GSW,Jimmy Butler,SF,SAC,22.542290,29.826465,-14.5,20.5,11.892233
19,GSW,De'Anthony Melton,PG,SAC,19.159517,25.026953,-14.5,11.5,7.262605
34,GSW,Brandin Podziemski,SG,SAC,20.837219,24.293935,-14.5,9.5,5.309089
52,GSW,Gary Payton II,SG,SAC,14.842323,13.476435,-14.5,5.5,8.397081
39,GSW,Moses Moody,SG,SAC,24.284437,24.669889,-14.5,9.5,7.125517
6,GSW,Stephen Curry,PG,SAC,30.981125,30.459158,-14.5,28.5,26.134680
92,GSW,Quinten Post,PF,SAC,21.869602,16.959641,-14.5,8.5,10.141533
91,GSW,Draymond Green,PF,SAC,24.200069,23.738193,-14.5,8.5,7.086185
120,GSW,Al Horford,C,SAC,16.550228,16.128933,-14.5,6.5,7.562954


,Team,Player,Pos,Opp,MP,MP_L5_avg,Spread,PTS_line,PTS_proj
3,DEN,Jamal Murray,PG,ATL,23.940649,37.691170,1.5,26.5,17.973438
72,DEN,Peyton Watson,SF,ATL,36.974709,35.329278,1.5,17.5,25.567533
83,DEN,Aaron Gordon,PF,ATL,20.925768,20.552643,1.5,16.5,10.676396
37,DEN,Tim Hardaway Jr.,SG,ATL,22.685877,30.588265,1.5,14.5,11.418423
46,DEN,Christian Braun,SG,ATL,31.698648,24.564873,1.5,12.5,9.487832


,Team,Player,Pos,Opp,MP,MP_L5_avg,Spread,PTS_line,PTS_proj
36,OKC,Jalen Williams,SG,MEM,34.689350,30.131147,-5.5,24.5,16.649227
48,OKC,Aaron Wiggins,SG,MEM,29.524626,23.782071,-5.5,16.5,8.752235
49,OKC,Isaiah Joe,SG,MEM,19.053110,15.264805,-5.5,12.5,5.560902
44,OKC,Ajay Mitchell,SG,MEM,31.418369,26.519915,-5.5,18.5,13.424884
99,OKC,Kenrich Williams,PF,MEM,18.608213,11.423841,-5.5,8.5,5.894490
54,OKC,Alex Caruso,SG,MEM,19.104782,17.174056,-5.5,6.5,5.051242
75,OKC,Luguentz Dort,SF,MEM,30.736071,25.052690,-5.5,9.5,9.712673


,Team,Player,Pos,Opp,MP,MP_L5_avg,Spread,PTS_line,PTS_proj
10,NOP,Jordan Poole,PG,WAS,21.293865,26.834264,-2.5,17.5,9.768254
81,NOP,Zion Williamson,PF,WAS,27.269873,30.465207,-2.5,26.5,19.722078
73,NOP,Trey Murphy III,SF,WAS,34.612583,31.490022,-2.5,20.5,24.997299
53,NOP,Micah Peavy,SG,WAS,15.232307,18.972519,-2.5,6.5,3.950470
12,NOP,Jeremiah Fears,PG,WAS,25.753304,26.362321,-2.5,14.5,12.480743
107,NOP,Derik Queen,C,WAS,26.340851,26.982380,-2.5,13.5,12.221751
125,NOP,Yves Missi,C,WAS,16.573950,20.282609,-2.5,5.5,5.615215


...

,Team,Player,Pos,Opp,MP,MP_L5_avg,Spread,PTS_line,PTS_proj
118,SAC,Maxime Raynaud,C,GSW,28.416058,32.583603,14.5,12.5,9.477695
84,SAC,DeMar DeRozan,PF,GSW,31.237789,32.062715,14.5,18.5,15.657830
7,SAC,Russell Westbrook,PG,GSW,27.759083,27.108784,14.5,14.5,17.196301
27,SAC,Zach LaVine,SG,GSW,30.723394,25.126786,14.5,20.5,18.228996
127,SAC,Precious Achiuwa,C,GSW,17.803028,19.542568,14.5,6.5,5.023685
8,SAC,Dennis Schroder,PG,GSW,21.022306,27.027110,14.5,11.5,10.413156
55,SAC,Nique Clifford,SG,GSW,19.765305,20.832966,14.5,5.5,4.794851


,Team,Player,Pos,Opp,MP,MP_L5_avg,Spread,PTS_line,PTS_proj
115,PHO,Mark Williams,C,NYK,23.321787,21.337439,1.5,11.5,8.551265
23,PHO,Devin Booker,SG,NYK,31.577562,30.802375,1.5,26.5,23.816547
65,PHO,Dillon Brooks,SF,NYK,29.275494,31.998530,1.5,19.5,17.217148
18,PHO,Jordan Goodwin,PG,NYK,20.501417,23.992790,1.5,8.5,10.725258
16,PHO,Collin Gillespie,PG,NYK,27.591234,28.824624,1.5,12.5,11.014903
26,PHO,Grayson Allen,SG,NYK,20.879576,22.913626,1.5,11.5,10.118511
76,PHO,Royce O'Neale,SF,NYK,27.782864,30.956196,1.5,9.5,8.695839
102,PHO,Oso Ighodaro,PF,NYK,20.299540,22.199490,1.5,5.5,5.369410


,Team,Player,Pos,Opp,MP,MP_L5_avg,Spread,PTS_line,PTS_proj
63,BOS,Jaylen Brown,SF,TOR,30.032644,35.476325,-10.5,28.5,26.169266
40,BOS,Anfernee Simons,SG,TOR,20.448523,27.087384,-10.5,13.5,11.793436
95,BOS,Sam Hauser,PF,TOR,23.986425,22.220096,-10.5,8.5,6.827394
28,BOS,Derrick White,SG,TOR,34.454891,36.112574,-10.5,18.5,16.936296
15,BOS,Payton Pritchard,PG,TOR,33.048523,33.441941,-10.5,16.5,15.243201
126,BOS,Luka Garza,C,TOR,18.013474,20.997892,-10.5,8.5,9.376067
119,BOS,Neemias Queta,C,TOR,25.010262,24.814613,-10.5,9.5,9.425366


,Team,Player,Pos,Opp,MP,MP_L5_avg,Spread,PTS_line,PTS_proj
77,BRK,Ziaire Williams,SF,LAC,16.683947,19.605493,4.5,6.5,8.274728
60,BRK,Michael Porter Jr.,SF,LAC,32.535412,32.230546,4.5,24.5,26.241852
121,BRK,Day'Ron Sharpe,C,LAC,19.241596,23.431131,4.5,6.5,7.695414
98,BRK,Noah Clowney,PF,LAC,27.360868,30.189937,4.5,11.5,12.202791


../tables/2025/gmday_preds_PTS.csv saved!


In [11]:
try: 
    generate_predictions('PRA')
except Exception as e:
    email('PRA', e)
    raise Exception(e)

98 rows


,Team,Player,Pos,Opp,MP,MP_L5_avg,Spread,PRA_line,PRA_proj
81,NOP,Zion Williamson,PF,WAS,27.269873,30.465207,-2.5,37.5,25.116512
10,NOP,Jordan Poole,PG,WAS,21.293865,26.834264,-2.5,23.5,13.552973
12,NOP,Jeremiah Fears,PG,WAS,25.753304,26.362321,-2.5,21.5,18.519539
107,NOP,Derik Queen,C,WAS,26.340851,26.982380,-2.5,27.5,26.066984
73,NOP,Trey Murphy III,SF,WAS,34.612583,31.490022,-2.5,30.5,31.664040


,Team,Player,Pos,Opp,MP,MP_L5_avg,Spread,PRA_line,PRA_proj
62,GSW,Jimmy Butler,SF,SAC,22.542290,29.826465,-14.5,31.5,20.256348
19,GSW,De'Anthony Melton,PG,SAC,19.159517,25.026953,-14.5,16.5,11.420475
6,GSW,Stephen Curry,PG,SAC,30.981125,30.459158,-14.5,37.5,32.752090
34,GSW,Brandin Podziemski,SG,SAC,20.837219,24.293935,-14.5,17.5,12.968635
91,GSW,Draymond Green,PF,SAC,24.200069,23.738193,-14.5,21.5,17.615986
92,GSW,Quinten Post,PF,SAC,21.869602,16.959641,-14.5,13.5,16.010515
52,GSW,Gary Payton II,SG,SAC,14.842323,13.476435,-14.5,9.5,11.901082
39,GSW,Moses Moody,SG,SAC,24.284437,24.669889,-14.5,13.5,11.394038
120,GSW,Al Horford,C,SAC,16.550228,16.128933,-14.5,12.5,13.673017


,Team,Player,Pos,Opp,MP,MP_L5_avg,Spread,PRA_line,PRA_proj
3,DEN,Jamal Murray,PG,ATL,23.940649,37.691170,1.5,40.5,30.367985
83,DEN,Aaron Gordon,PF,ATL,20.925768,20.552643,1.5,24.5,16.841286
72,DEN,Peyton Watson,SF,ATL,36.974709,35.329278,1.5,26.5,32.506523
37,DEN,Tim Hardaway Jr.,SG,ATL,22.685877,30.588265,1.5,20.5,15.856602
46,DEN,Christian Braun,SG,ATL,31.698648,24.564873,1.5,20.5,17.335999


,Team,Player,Pos,Opp,MP,MP_L5_avg,Spread,PRA_line,PRA_proj
48,OKC,Aaron Wiggins,SG,MEM,29.524626,23.782071,-5.5,24.5,14.472889
36,OKC,Jalen Williams,SG,MEM,34.689350,30.131147,-5.5,37.5,27.791098
44,OKC,Ajay Mitchell,SG,MEM,31.418369,26.519915,-5.5,28.5,22.041548
54,OKC,Alex Caruso,SG,MEM,19.104782,17.174056,-5.5,12.5,8.180182
75,OKC,Luguentz Dort,SF,MEM,30.736071,25.052690,-5.5,15.5,16.175814


...

,Team,Player,Pos,Opp,MP,MP_L5_avg,Spread,PRA_line,PRA_proj
66,LAL,LeBron James,SF,MIL,31.565365,31.869368,-3.5,35.5,38.248699
47,LAL,Marcus Smart,SG,MIL,29.366116,31.652022,-3.5,15.5,18.092941
5,LAL,Luka Doncic,PG,MIL,34.526794,36.960958,-3.5,52.5,52.242069


,Team,Player,Pos,Opp,MP,MP_L5_avg,Spread,PRA_line,PRA_proj
13,ORL,Anthony Black,PG,PHI,30.866930,31.434533,3.5,26.5,24.102583
82,ORL,Paolo Banchero,PF,PHI,33.807343,34.743535,3.5,37.5,36.438652
24,ORL,Desmond Bane,SG,PHI,32.163277,32.769899,3.5,28.5,28.112589


,Team,Player,Pos,Opp,MP,MP_L5_avg,Spread,PRA_line,PRA_proj
29,WAS,Tre Johnson,SG,NOP,25.513710,27.821514,2.5,20.5,18.464598
74,WAS,Kyshawn George,SF,NOP,30.831886,31.396171,2.5,22.5,24.413422
78,WAS,Khris Middleton,SF,NOP,24.335001,20.535755,2.5,17.5,18.113466
35,WAS,Bilal Coulibaly,SG,NOP,26.887815,28.825853,2.5,21.5,21.070379


,Team,Player,Pos,Opp,MP,MP_L5_avg,Spread,PRA_line,PRA_proj
60,BRK,Michael Porter Jr.,SF,LAC,32.535412,32.230546,4.5,35.5,36.102070
121,BRK,Day'Ron Sharpe,C,LAC,19.241596,23.431131,4.5,16.5,16.657980
98,BRK,Noah Clowney,PF,LAC,27.360868,30.189937,4.5,17.5,17.350653


../tables/2025/gmday_preds_PRA.csv saved!
